## Simulating observations with MUSTANG-2

MUSTANG-2 is a bolometric array on the [Green Bank Telescope](https://en.wikipedia.org/wiki/Green_Bank_Telescope). In this notebook we simulate an observation of a galactic cluster.

In [ ]:
import maria

map_filename = maria.io.fetch("maps/small_cluster.h5")

input_map = maria.map.load(
    nu=930e9,
    filename=map_filename,  # filename
    width=6 / 60,  # width in degrees
    center=(144, 12),  # position in the sky
    frame="ra_dec",
    units="Jy/pixel",  # units of the input map
)

input_map.plot()

In [ ]:

plan = maria.get_plan(
    scan_pattern="daisy",  # scanning pattern
    scan_options={"radius": 2 / 60, "speed": 0.5 / 60},  # in degrees
    duration=1200,  # integration time in seconds
    sample_rate=50,  # in Hz
    scan_center=(144, 12),  # position in the sky
    frame="ra_dec",
)

plan.plot()

In [ ]:
instrument = maria.get_instrument("MUSTANG-2")
print(instrument)
instrument.plot()

In [ ]:
sim = maria.Simulation(
    instrument,
    plan=plan,
    site="green_bank",
    map=input_map,
    atmosphere="2d",
)

print(sim)

In [ ]:
tod = sim.run()
tod.plot()

In [ ]:
from maria.mappers import BinMapper

mapper = BinMapper(
    center=(144, 12),
    frame="ra_dec",
    width=6 / 60,
    height=6 / 60,
    resolution=0.03 / 60,
    tod_preprocessing={
        "window": {"name": "hamming"},
        "remove_modes": {"modes_to_remove": [0]},
        "remove_spline": {"knot_spacing": 30, "remove_el_gradient": True},
    },
    map_postprocessing={
        "gaussian_filter": {"sigma": 1},
        "median_filter": {"size": 1},
    },
    units="uK_RJ",
)

mapper.add_tods(tod)
output_map = mapper.run()

In [ ]:
output_map.plot()
output_map.to_fits("/tmp/simulated_mustang_map.fits")